## QKD Network Simulation

In [1]:
from qiskit import QuantumCircuit, Aer, execute
from numpy.random import randint, normal, choice, shuffle
import numpy as np
import time
import os
from threading import Thread, Event, Barrier
qasm_sim = Aer.get_backend('qasm_simulator')

Defining support functions and classes

In [2]:
class qubit:
    def __init__(self, ID, bit, basis, stream):
        self.basis = basis
        self.stream = stream
        self.id = ID
        self.bit = bit

Alice Substation: Generates the QKD photons based on a random bit and basis generator and outputs 2 objects- sent_qubits (qubits sent to bob) and alice_buffer (basis and ID information of the photons sent)

In [3]:
def alice_station(num_bits, stream, bits = None, qubit_id = None):
    if bits == None:
        bits = randint(2, size=num_bits)
        
    if qubit_id == None:
        qubit_id = [os.urandom(4) for i in range(num_bits)] 
    bases = randint(2, size=num_bits)
    sent_qubits = {}
    alice_buffer = {}
    for i in range(num_bits):
        qc = QuantumCircuit(1,1)
        if bits[i]: qc.x(0)
        qc.h(0)
        if bases[i]: qc.rz(np.pi/2, 0)  
        qc.barrier()
        
        sent_qubits[qubit_id[i]] = qc
        alice_buffer[qubit_id[i]] = qubit(qubit_id[i], bits[i], bases[i], stream)
        
    return sent_qubits, alice_buffer

Route photons based on the path provided and perform an attack if any link in the path is attacked

In [4]:
def transmit_over_channel(messages, path, alice, bob, stream, conn_no, max_rate = None):
    attenuation = 0.1
    noise = normal(0, 0.25, (3,))
    if max_rate == None:
        max_rate = path[1]['maxRate']
    for i in range(1, len(path)-1):

        if type(path[i]) == int:
            for qubit_id in messages.keys():
                router_logs[path[i]].write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ {alice} --> {bob}\n")
                router_logs[path[i]].flush()
        else:
#             if path[i]['maxRate'] < max_rate:
#                 max_rate = path[i]['maxRate']
                
            for qubit_id in messages.keys():
                if not messages[qubit_id] == None:
                    messages[qubit_id].ry(np.pi/25, 0)
                
                if not choice([0,1], 1, p=[attenuation, 1-attenuation]):
                    messages[qubit_id] = None
                

            if path[i]['isEve']:   
                temp = messages.copy()
                messages.clear()
                messages.update(eve(temp, stream))
            if path[i]['isDarth']:
                darth(messages)
                break
    reserved_resources[conn_no] = (len(path)//2)*max_rate
#     return max_rate

Implementation of attacks performed in the transmit_over_channel function

In [5]:
def darth(sent_qubits):
    for qubit_id in sent_qubits.keys():
        sent_qubits[qubit_id] = None
        
def eve(sent_qubits, stream):
    measured_data = bob_station(sent_qubits, stream)
    bits = generate_key(measured_data)
    IDs = []
    for i in measured_data:
        IDs.append(i.id)
    re_encoded_data, _ = alice_station(len(bits), stream, bits, IDs)
    return re_encoded_data

Bob substation, measures the qubits using a random basis and stores the measurement outcome, basis of measurement, stream of QKD and ID of photon in Bob's buffer

In [6]:
def bob_station(messages, stream):
    bob_buffer = []
    bases = randint(2, size=len(messages))
    
    for q, (qubit_id, photon) in enumerate(messages.items()):
        if not photon == None:
            if int(bases[q]): photon.rz(-np.pi/2, 0)
            photon.h(0)
            photon.measure(0,0)
            result = execute(photon, qasm_sim, shots = 1).result()
            
            bob_buffer.append(qubit(qubit_id, int(list(result.get_counts().keys())[0]), bases[q], stream))
    return bob_buffer

Post Processor functions:. Contains the following functions:<br>
1. alice_post_processing: Performs post processing at Alice's end. Runs as individual thread, synchronized with bob_post_processing using events
2. bob_post_processing: Performs post processing at Bob's end.
3. generate_key: Extracts key bits from a list of qubit objects.

In [7]:
def generate_key(qubits):
    keys = []
    for i in qubits:
        keys.append(i.bit)
    return keys

def alice_post_processing(alice_buffer, alice_key, number_streams, sent_photons_stream, conn_no, conn_id, gtg_flag, attack_flag, classical_channel, paths, paused_paths):
    empty_streams = [0 for i in range(number_streams)]
    print(f'alice_{conn_id}.txt')
    alice_logs = open(f'alice_{conn_id}.txt', 'w')
    while True:
        gtg_flag.wait()
        secondary_buffer = []
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ alice_buffer --> {len(alice_buffer)}\n")
        alice_logs.flush()
        secondary_buffer = alice_buffer.copy()
        alice_buffer.clear()       
        
        
        sent_photons_by_stream = sent_photons_stream.copy()
        sent_photons_stream.clear()
        sent_photons_stream.extend(empty_streams)
        
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ sent_photons_stream --> {sent_photons_by_stream}\n")
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ secondary_buffer --> {len(secondary_buffer)}\n")
        alice_logs.flush()
        while gtg_flag.is_set():
            pass
        
        
        good_bits = []
        final_ids = []
        for i in classical_channel:
            recieved_id = i[0]
            recieved_basis = i[1]
            try:                                                                                     
                if secondary_buffer[recieved_id].basis == recieved_basis:                                
                    good_bits.append(secondary_buffer[recieved_id])                                      
                    final_ids.append(recieved_id)
                del secondary_buffer[recieved_id]                                                        
            except:                                                                                  
                pass
            
        unacknowledged_photons_stream = [0 for i in range(number_streams)]                           
        for i in secondary_buffer.values():                                                              
            unacknowledged_photons_stream[i.stream] += 1 
        
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ unacknowledged_photons_stream --> {unacknowledged_photons_stream}\n")
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ secondary_buffer --> {len(secondary_buffer)}\n")
        alice_logs.flush()
        
        classical_channel.clear()
        classical_channel.extend(final_ids)
        gtg_flag.set() 
        darth_targets, _ = check_darth(unacknowledged_photons_stream, sent_photons_by_stream, paused_paths)
            
        for i in darth_targets:
            paused_paths.add(i)
            darth_resolver = Thread(target = reslove_darth, args = (paused_paths, i, paths[i], conn_no+i))
            darth_resolver.start()
    
        alice_key_sample = good_bits[:len(good_bits)//3]
        
        while gtg_flag.is_set():
            pass
        
        eve_targets, stream_qber = check_eve(alice_key_sample, classical_channel, number_streams, paused_paths, paths)
        
        for i in eve_targets:
            paused_paths.add(i)
            eve_resolver = Thread(target = reslove_eve, args = (paused_paths, i, paths[i], conn_no+i))
            eve_resolver.start()
        
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ darth_targets --> {darth_targets}\n")
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ eve_targets --> {eve_targets}\n")
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ paused_paths --> {paused_paths}\n")
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ stream_qber --> {stream_qber}\n")
        alice_logs.flush()
        
        if not (darth_targets == [] or eve_targets == []):
            attack_flag.set()
        gtg_flag.set()
        
        
        if attack_flag.is_set():
            try:
                for i in range(len(good_bits)):
                    if good_bits[i].stream in paused_paths:
                        del good_bits[i]
            except:
                pass
        old_key_length = len(alice_key)
        alice_key.extend(good_bits)
        new_key_length = len(alice_key)
        
        key_rates[conn_id] = new_key_length - old_key_length
        
        alice_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ alice_key_size --> {len(generate_key(alice_key))}\n\n")
        alice_logs.flush()
        gtg_flag.clear()   
    
    
def bob_post_processing(bob_buffer, bob_key, conn_no, gtg_flag, attack_flag, classical_channel, paused_paths, clock):
    bob_logs = open(f'bob_{conn_no}.txt', 'w')
    while True:
        clock.wait()
        gtg_flag.set()
        secondary_buffer = bob_buffer.copy()
        bob_buffer.clear()
        
        bob_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ bob_buffer_size --> {len(secondary_buffer)}\n")
        bob_logs.flush()
        classical_channel.clear()
        for qubit in secondary_buffer:                                                                     
            classical_channel.append((qubit.id, qubit.basis))
            
        gtg_flag.clear()
        gtg_flag.wait()
    
        good_bits = []
        for i in classical_channel:
            while not i == secondary_buffer[0].id:                                               
                del secondary_buffer[0]                                                                   
            good_bits.append(secondary_buffer[0])
        
        bob_logs.flush()
        
        classical_channel.clear()
        classical_channel.extend(good_bits[:len(good_bits)//3])
        
        gtg_flag.clear()
        gtg_flag.wait()
        
        if attack_flag.is_set():
            try:
                for i in range(len(good_bits)):
                    if good_bits[i].stream in paused_paths:
                        del good_bits[i]
            except:
                pass
            
        bob_key.extend(good_bits)

        bob_logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ bob_key_len --> {len(generate_key(bob_key))}\n\n")
        bob_logs.flush()

Implementation of functions for detecting DoS (check_darth) and Man in the Middle (check_eve) attacks used in post processing functions. 

In [8]:
def check_darth(unacknowledged_photons_stream, sent_photons_stream, paused_paths):
    photon_loss = []
    attacked_streams = []
    for i in range(len(sent_photons_stream)):
        try:
            loss = (unacknowledged_photons_stream[i]/sent_photons_stream[i])*100
            if loss > 90 and i not in paused_paths:
                photon_loss.append(loss)
                attacked_streams.append(i)
        except:
            pass
    return attacked_streams, photon_loss
            
def check_eve(alice_key_sample, bob_key_sample, num_streams, paused_paths, paths):
    stream_keys = [0 for i in range(num_streams)]
    stream_errors = [0 for i in range(num_streams)]
    stream_qber = [0 for i in range(num_streams)]
    attacked_streams = []
    
    for i, j in zip(alice_key_sample, bob_key_sample):
        stream_keys[i.stream] += 1
        if not i.bit == j.bit:
            stream_errors[i.stream] += 1

    for i in range(num_streams):
        try:
            stream_qber[i] = (stream_errors[i]/stream_keys[i])*100
            if (stream_qber[i]) > (len(paths[i])//2)*9 and i not in paused_paths:
                attacked_streams.append(i)
        except:
            pass
    
    return attacked_streams, stream_qber

Implementations of functions to find the attacked nodes, find a new path bypassing that node and resume key exchange over the stream. 

In [9]:
def reslove_eve(paused_paths, path_no, path, conn_no):
    print("Eavesdropping attack detected")
    last_node_before_attack = None
    original_alice = path[0]
    original_bob = path[-1]
    n = len(path)
    max_photons_to_find = 2000
    r = 1.3
    a = max_photons_to_find * (1 - r) // (1 - r**(n//2))
    photon_distribution = [0 for i in range(n//2)]
    
    for i in range(len(photon_distribution)):
        photon_distribution[i] = int(a * r**(len(photon_distribution) - i - 1))
    j = 0
    print('photon_distribution', photon_distribution)
    for i in range(n-1):
        alice_key_sample = []
        bob_key_sample = []
        if type(path[i]) == int:
            alice = path[i]
            bob = path[-1]
            sent_qubits, alice_buffer_data = alice_station(photon_distribution[j], 0)

            transmit_over_channel(sent_qubits, path[i:], alice, bob, 0, conn_no)

            bob_buffer_data = bob_station(sent_qubits, 0)
            
            for k in bob_buffer_data:
                if alice_buffer_data[k.id].basis == k.basis:
                    alice_key_sample.append(alice_buffer_data[k.id])
                    bob_key_sample.append(k)
            attacked_streams, qber = check_eve(alice_key_sample, bob_key_sample, 1, [], [path])
            print('qber:',path[i], ':', qber)
            if not attacked_streams == []:
                last_node_before_attack = path[i]
            j+=1
    if last_node_before_attack == None:
        print("attack not found")
        paused_paths.remove(path_no)
        return
    i = 0
    while i < n-1:
        if type(path[i]) == int:
            if path[i] == last_node_before_attack:
                i+=2
                continue
        else:
            path[i]['isOccupied'] = 0
        i+=1
    print('last_node_before_attack', last_node_before_attack)
    updated_path = find_path(original_alice, original_bob)
    if updated_path == None:
        print('no new path found')
        return 
    print('new path found')
    path.clear()
    path.extend(updated_path)
    
    paused_paths.remove(path_no)                

def reslove_darth(paused_paths, path_no, path, conn_no):
    print("DoS attack detected")
    last_node_before_attack = None
    original_alice = path[0]
    original_bob = path[-1]
    n = len(path)
    max_photons_to_find = 1000
    r = 1.3
    a = max_photons_to_find * (1 - r) // (1 - r**(n//2))
    photon_distribution = [0 for i in range(n//2)]
    
    for i in range(len(photon_distribution)):
        photon_distribution[i] = int(a * r**(len(photon_distribution) - i - 1))
    j = 0
    print('photon_distribution', photon_distribution)
    for i in range(n-1):
        if type(path[i]) == int:
            alice = path[i]
            bob = path[-1]
            sent_qubits, alice_buffer_data = alice_station(photon_distribution[j], 0)
            num_sent_photons = len(sent_qubits)
            transmit_over_channel(sent_qubits, path[i:], alice, bob, 0, conn_no)

            bob_buffer_data = bob_station(sent_qubits, 0)
            
            attacked_streams, loss = check_darth([len(alice_buffer_data)-len(bob_buffer_data)], [len(alice_buffer_data)], [])
            print('loss:',path[i], ':', loss)
            if not attacked_streams == []:
                print(path[i], 'attacked')
                last_node_before_attack = path[i]
            j+=1
    
    if last_node_before_attack == None:
        print("attack not found")
        paused_paths.remove(path_no)
        return
    
    i = 0
    while i < n-1:
        if type(path[i]) == int:
            if path[i] == last_node_before_attack:
                i+=2
                continue
        else:
            path[i]['isOccupied'] = 0
        i+=1
    print('last_node_before_attack', last_node_before_attack)
    updated_path = find_path(original_alice, original_bob)
    if updated_path == None:
        print('no new path found')
        return 
    print('new path found: ', updated_path)
    path.clear()
    path.extend(updated_path)
    
    paused_paths.remove(path_no)

Routing functions. Contains the following functions:
1. djikstras: Finds a path of minimum cost from a given source to destination in the router
2. find_min: Support function for djikstras function
3. find_path: Creates weight matrix based on the network topology and returns a path from a given source to destination

In [10]:
def djikstras(weight_matrix, source, n):
    visited = [0 for i in range(n)]
#     temp_bw = [0 for i in range(n)]
#     max_bw = sum([sum(i) for i in weight_matrix])
    bw = [0 for i in range(n)]
    routing_table = [0 for i in range(n)]
    bw[source] = float('inf')

    for i in range(n):
        u = find_max(bw, visited, n)
        if u == -1:
            break
        visited[u] = 1
        for j in range(n):
            if bw[j] < min(weight_matrix[u][j], bw[u]):
                routing_table[j] = u
                bw[j] = 0.8*min(weight_matrix[u][j], bw[u])
    return routing_table, bw

def find_max(bw, visited, n):
    max_index = -1
    max_bw = 0
    for i in range(n):
        if bw[i] > max_bw and visited[i] == 0:
            max_bw=bw[i]
            max_index=i
#             print('max_bw = ',max_bw, 'max_index = ', max_index)
    return max_index

def find_path(alice, bob, requested_rate):
    destination_router = None
    source_router = None
    max_rate = float('inf')
#     print("host_links:")
#     for i in host_links:
#         print(i)
#     min_source_rate = 0
#     min_dest_rate = 0
    for i in host_links:
        if i['host'] == alice and i['maxRate']>0:
            source_router = i['router']
            source_link = i
            max_rate = min(requested_rate, i['maxRate'], max_rate)
#             if i['maxRate'] < max_rate:
#                 max_rate = i['maxRate'] 
#             min_source_rate = i['maxRate']
            
        if i['host'] == bob and i['maxRate']>0:
            destination_router = i['router']
            destination_link = i
            max_rate = min(requested_rate, i['maxRate'], max_rate)
#             if i['maxRate'] < max_rate:
#                 max_rate = i['maxRate']
#             min_dest_rate = i['maxRate']
            
    if destination_router == None or source_router == None:
        print("destination_router", destination_router, "source_router", source_router, '\n')
        return None
#     else:
#         print("found host link")
    
    weight_matrix = [[float('inf') if i == j else 0 for i in range(num_routers)] for j in range(num_routers)]
    for i in router_links:
        if i['maxRate']>0:
            weight_matrix[i['node_a']][i['node_b']] = i['maxRate']
            weight_matrix[i['node_b']][i['node_a']] = i['maxRate']
#     print("weight_matrix:")
#     for i in weight_matrix:
#         print(i)
    
#     print("destination: ", destination_router, "source: ", source_router)
    trail, cost = djikstras(weight_matrix, source_router, num_routers)
#     print('trail: ', trail, 'cost: ', cost)
#     print("rexcieved trail = ", trail)
    if cost[destination_router] == 0:
        print("dist of host infinite\n")
        return None
    
    current_node = destination_router
    path = []
    while current_node != source_router:
#         print('yo')
        path.insert(0, current_node)
        node_1 = current_node
        current_node = trail[current_node]
        node_2 = current_node
        for i in router_links:
            if ((i['node_a'] == node_1 and i['node_b'] == node_2) or (i['node_a'] == node_2 and i['node_b'] == node_1)) and i not in path:
    #                 i['isOccupied'] = 1
                path.insert(0, i)
                max_rate = min(requested_rate, i['maxRate'], max_rate) 
                break
    
#     minimum_rate = float('inf')
#     for i in path:
#         minimum_rate = min(minimum_rate, required_rate, i['maxRate'])
    
    
#     print("path: ", path)
    path.insert(0, source_router)
#     for i in host_links:
#         if (i['host'] == alice and i['router'] == source_router) and i['isOccupied'] == 0:
#             i['isOccupied'] = 1
#             path.insert(0, i)
            
#         if (i['host'] == bob and i['router'] == destination_router) and i['isOccupied'] == 0:
#             i['isOccupied'] = 1
#             path.append(i)
            
#     source_link['isOccupied'] = 1
    path.insert(0, source_link)
    
#     destination_link['isOccupied'] = 1
    path.append(destination_link)
    
    path.insert(0, alice)
    path.append(bob)
    
    for i in path:
        if not type(i) == int:
            i['maxRate'] = 0 # for bandwidth sharing algorithm, replace this with i['maxRate'] -= max_rate
    
#     print("returned paths: ", path)
    print(f"({path[1]['host']},{path[1]['router']})", end = ', ')
    for i in path[2:-2]:
        if not type(i) == int:
            print(f"({i['node_a']},{i['node_b']})", end = ', ')
    print(f"({path[-2]['router']},{path[-2]['host']})", end = ', ')
    print('max_rate: ', max_rate)
    return path, max_rate

Functions to simulate key exchange:
1. run_parallel_qkd: initializes variables and threads for key exchange and post-processing the keys
2. qkd_stream: Simulates the process of generating QKD photons at Alice's end, transmitting them through the channel and measuring them at Bob's end

In [11]:
def run_parallel_qkd(number_streams, alice, bob, conn_no, conn_id, max_rate):    
    alice_buffer = {}
    bob_buffer = []
    alice_key = []
    bob_key = []
    gtg_flag = Event()
    attack_flag = Event()
    attack_flag.clear()
    classical_channel = []
    sent_photons_stream = []
    paused_paths = set()
    paths = []
    assigned_rates = []
#     enabled_streams = 0
#     atleast_one = False    
#     print("inside run_parallel_qkd")
    sent_photons_stream = [0 for i in range(number_streams)]
    for i in range(number_streams):
#         path, assigned_rate
        path_info = find_path(alice, bob, max_rate[i])
        
        if not path_info == None:
            paths.append(path_info[0])
#             print('path\n', path)
            assigned_rates.append(path_info[1])
#         else:
#             print("path not found")
#             atleast_one = True

#             stream = Thread(target = qkd_stream, args = (i, path, alice, bob, alice_buffer, bob_buffer, sent_photons_stream, paused_paths, max_rate[i], clock, conn_no+i))
#             stream.start()
#             enabled_streams += 1
#     print("checked for paths")
    
    clock = Barrier(len(paths)+1)
    if len(paths) > 0:
        for i in range(len(paths)):
            stream = Thread(target = qkd_stream, args = (i, paths[i], alice, bob, alice_buffer, bob_buffer, sent_photons_stream, paused_paths, assigned_rates[i], clock, conn_no+i))
            stream.start()
            
        bob_post = Thread(target = bob_post_processing, args = (bob_buffer, bob_key, conn_no, gtg_flag, attack_flag, classical_channel, paused_paths, clock))
        bob_post.start()
        alice_post_processing(alice_buffer, alice_key, number_streams, sent_photons_stream, conn_no, conn_id, gtg_flag, attack_flag, classical_channel, paths, paused_paths)
    else:
        print('No Paths available')

def qkd_stream(stream, path, alice, bob, alice_buffer, bob_buffer, sent_photons_stream, paused_paths, rate, clock, conn_no):
#     while True:
    for _ in range(3):
        start = time.time()
        if stream not in paused_paths:
            sent_photons_stream[stream] += rate
            sent_qubits, alice_buffer_data = alice_station(rate, stream)
            alice_buffer.update(alice_buffer_data)

            transmit_over_channel(sent_qubits, path, alice, bob, stream, conn_no, rate)
            
            bob_buffer_data = bob_station(sent_qubits, stream)
            bob_buffer.extend(bob_buffer_data)
            clock.wait()
            end = time.time()
            print(f"{conn_no} time: %.2f"%(end-start))
#             time.sleep(10)
            time.sleep(5)
        else:
            time.sleep(0.5)

Defining topology of the network and router logs

In [12]:
def initialise_network():
#     hosts = [6,7,8,9]
#     routers = [0 for i in range(6)]

    global router_links
    global host_links
    global num_routers
    global router_logs
    global reserved_resources
    global key_rates
    
    
    reserved_resources = []
    key_rates = []
#     num_routers = len(routers)
    num_routers = 14

    router_links = [{'node_a':0, 'node_b':1, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':1, 'node_b':2, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':1, 'node_b':3, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':0, 'node_b':2, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':0, 'node_b':7, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':5, 'node_b':2, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':3, 'node_b':4, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':3, 'node_b':10, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':4, 'node_b':5, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':4, 'node_b':6, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':6, 'node_b':7, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':5, 'node_b':9, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':5, 'node_b':12, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':9, 'node_b':8, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':12, 'node_b':13, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':13, 'node_b':8, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':12, 'node_b':11, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':8, 'node_b':11, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':10, 'node_b':13, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':10, 'node_b':11, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                    {'node_a':7, 'node_b':8, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0}]

#     router_links = [{'node_a':12, 'node_b':5, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
#                     {'node_a':12, 'node_b':13, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0}]
                    
                    
    for i in range(len(router_links)):
        router_links[i]['maxRate'] = 3*choice(list(range(800, 2000)))
#         print(router_links[i])
        
    host_links = [{'router':2, 'host':34, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':2, 'host':33, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':2, 'host':32, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':5, 'host':31, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':5, 'host':30, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':5, 'host':29, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':9, 'host':28, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':9, 'host':27, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':9, 'host':26, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':12, 'host':25, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':13, 'host':24, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':13, 'host':23, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':11, 'host':22, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':11, 'host':21, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':11, 'host':20, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':10, 'host':19, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':10, 'host':18, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':0, 'host':17, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':0, 'host':16, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':0, 'host':15, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':1, 'host':14, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':1, 'host':35, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':10, 'host':36, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':13, 'host':37, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':12, 'host':38, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},
                  {'router':12, 'host':39, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'occupancy':0},]
#                   {'router':1, 'host':40, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':10000},]
    for i in range(len(host_links)):
        host_links[i]['maxRate'] = choice(list(range(800, 2000)))
    
    for i in range(5*len(host_links)):
        host_links.append(host_links[i].copy())
    
#     print("\nhost_links:")
#     for i in host_links:
#         print(i)
    router_logs = [open(f'router_{i}.txt', 'w') for i in range(num_routers)]
    
    
def network_analyzer(record):
    global reserved_resources
    global key_rate
    global host_links
    global router_links
    logs = open('FCFS_network_anlyzer.csv', 'a')
    logs.write(f'\n{time.strftime("%d/%m %H:%M:%S", time.localtime())} @ analyzer initialized\n\n')
    logs.flush()
    total_resources = 0
    for i in router_links:
        total_resources += i['maxRate']
    for i in host_links:
        total_resources += i['maxRate']
#     print("in here")
    while True:
        resource_utilization = reserved_resources
#         reserved_resources = [0 for i in range(len(reserved_resources))]
        try:
            record.wait()
            average_key_rate = sum(key_rates)/len(key_rates)
            net_resource_utilization = sum(resource_utilization)/total_resources
            logs.write(f'average_key_rate:,{average_key_rate},net_resource_utilization:,{net_resource_utilization},key_rates:,"{key_rates}",resource_utilization:,"{resource_utilization}"\n')

#             logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ resource_utilization --> {resource_utilization}\n\n")
#             logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ average_resource_utilization --> {sum(resource_utilization)/len(resource_utilization)}\n\n")
#             logs.write(f"{time.strftime('%d/%m %H:%M:%S', time.localtime())} @ key_rates --> {key_rates}\n\n")
        except:
            pass
        logs.flush()

Main function to execute the simulation
Running the simulator: 
1. Restart the kernel and execute all cells above. 
2. Execute the cell below. This will create router logs for all the routers in the topology in the directory of the notebook
3. We need to observe the router logs in real time. This can be done using:
    * If you are using a Linux Machine, use: ```tail -f <path to file>``` 
    * If you are using a Windows Machine, use: ```Get-content <path to file> -Wait ```
4. Follow the menu to start the simulator
    * Once a connection is initiated, 2 more logs are created- Alice and bob. These can also be observed in real time using the commands above

In [ ]:
def main(record):
    initialise_network()
    Thread(target = network_analyzer, args = (record,)).start()
#     print('yo')
    menu = {
        '1' : 'create_new_connection()',
        '2' : 'attack()',
        '3' : 'sys.exit()'
    }
    while True:
        print("Pick an option:")
        for i, j in menu.items():
            print(i, ':', j)
        eval(menu.get(input(), "print('wrong key')"))
        time.sleep(10)
        record.wait()
        print('reserved_resources', reserved_resources, 'key_rates', key_rates)
        time.sleep(1)

def main_automatic(is_multiple, record):
    initialise_network()
    Thread(target = network_analyzer, args = (record,)).start()
    if is_multiple:
        num_streams = lambda : choice([1,2,3])
    else:
        num_streams = lambda : 1
    
    hosts = list(range(14,40))
    for _ in range(3):
        alice_list = list(choice(hosts, size = len(hosts)//2, replace = False))
        bob_list = []
        for j in hosts:
            if j not in alice_list:
                bob_list.append(j)

        for i in range(len(alice_list)):
            print(type(alice_list[i]))
            streams = num_streams()
            create_new_connection([int(alice_list[i]), int(bob_list[i]), streams, [choice(list(range(800, 2000))) for i in range(streams)]])        
            time.sleep(15)
            record.wait()
    
#     alice_list = list(range(14, 25))
#     bob_list = list(range(25, 36))
#     shuffle(alice_list)
#     shuffle(bob_list)
    
#     for i in range(11):
#         streams = num_streams()
#         create_new_connection([alice_list[i], bob_list[i], streams, [choice(list(range(800, 2000))) for i in range(streams)]])        
# #         time.sleep(50)
#         time.sleep(20)
#         record.wait()
#         time.sleep(5)
        
def create_new_connection(params = None):
    global reserved_resources
    if params == None:
        alice = int(input("Enter Alice index"))
        bob = int(input("Enter Bob index"))
        num_streams = int(input("Enter number of streams"))
        max_rate = [0 for i in range(num_streams)]
        for i in range(num_streams):
            max_rate[i] = int(input(f"Enter rate of stream {i}"))
    else:
        alice = params[0]
        bob = params[1]
        num_streams = params[2]
        max_rate = params[3]
    print("\nnumber of streams:", num_streams, ", source:", alice, ", dest:", bob, ", max_rate:", max_rate)
    conn = Thread(target = run_parallel_qkd, args = (num_streams, alice, bob, len(reserved_resources), len(key_rates), max_rate))
    conn.start()
    reserved_resources.extend([0 for i in range(num_streams)])
    key_rates.append(0)
    

def attack():
    ch = int(input("attack router links (enter 0) or host links (enter 1)"))
    if ch:
        print("host links are:")
        for i in range(len(host_links)):
            print(i, ':', host_links[i]['host'], '<-->', host_links[i]['router'])
        attacked_link_index = int(input("input index of link to attack"))
        attacked_link = host_links[attacked_link_index]
    else:
        print("router links are:")
        for i in range(len(router_links)):
            print(i, ':', router_links[i]['node_a'], '<-->', router_links[i]['node_b'])
        attacked_link_index = int(input("input index of link to attack"))
        attacked_link = router_links[attacked_link_index]
    
    attack_type = int(input("perform Eavesdropping attack (Enter 1) or DoS attack (Enter 0)"))
    
    if attack_type:
        attacked_link['isEve'] = 1
    else:
        attacked_link['isDarth'] = 1


option = int(input("manual(0) or automatic(1)?"))
# option = 1
# is_multiple = 0
record = Barrier(2)
# for i in range(4):
if option:
    is_multiple = int(input('is_multiple'))
    main_automatic(is_multiple, record)
else:
    main(record)
    

manual(0) or automatic(1)? 1
is_multiple 1


<class 'numpy.int32'>

number of streams: 3 , source: 26 , dest: 16 , max_rate: [1717, 1553, 1354]
(26,9), (9,8), (7,8), (0,7), (0,16), max_rate:  822
(26,9), (5,9), (5,2), (1,2), (0,1), (0,16), max_rate:  822
dist of host infinite

alice_0.txt
0 time: 4.12
1 time: 4.10
1 time: 3.470 time: 3.45

<class 'numpy.int32'>

number of streams: 3 , source: 15 , dest: 18 , max_rate: [1455, 1994, 1984]
dist of host infinite

dist of host infinite

dist of host infinite

No Paths available
1 time: 3.370 time: 3.41

<class 'numpy.int32'>

number of streams: 1 , source: 24 , dest: 19 , max_rate: [910]
(24,13), (10,13), (10,19), max_rate:  806
alice_2.txt
6 time: 1.96


**Ansh Singal**<br>
3rd year UG CSE<br>
RV College of Engineering<br>

In [2]:
from threading import Thread, Barrier, Event, Lock
import time
def a(var, var2):
    for i in range(10):
        time.sleep(1)
        print(var2)
#         var.flush()
#     event.set()
    
def b(var, var2):
#     for i in range(10):
    time.sleep(4)
    var.acquire()
    for i in range(5):
        print("in b", var2)
#     time.sleep(3)
    var.release()
#         var.write("hello from b\n")
#         var.flush()
#     event.set()
    
def c(var):
    for i in range(2):
        var.wait()
        print("C got it")
    
var = Lock()
var2 = 'a'
# var = open('test.txt', 'w')
# event = Event()
t_a = Thread(target = a, args = (var,var2))
t_b = Thread(target = b, args = (var,var2))
# t_c = Thread(target = c, args = (var,))
t_a.start()
t_b.start()
# t_c.start()
t_a.join()
t_b.join()
# t_c.join()

a
a
a
in b a
in b a
in b a
in b a
in b a
a
a
a
a
a
a
a


In [1]:
from threading import Thread
import time
def a():
    for i in range(5):
        print('hi')
Thread(target = a).start()
time.sleep(2)
print(type(b))

hi
hi
hi
hi
hi


NameError: name 'b' is not defined

In [ ]:
host_links = [{'router':2, 'host':34, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2000},
              {'router':2, 'host':33, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':3000},
              {'router':2, 'host':32, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1500},
              {'router':5, 'host':31, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2500},
              {'router':5, 'host':30, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1600},
              {'router':5, 'host':29, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1000},
              {'router':9, 'host':28, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2000},
              {'router':9, 'host':27, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2300},
              {'router':9, 'host':26, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1500},
              {'router':12, 'host':25, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1700},
              {'router':13, 'host':24, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1400},
              {'router':13, 'host':23, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':750},
              {'router':11, 'host':22, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1000},
              {'router':11, 'host':21, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1500},
              {'router':11, 'host':20, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1750},
              {'router':10, 'host':19, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1600},
              {'router':10, 'host':18, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2600},
              {'router':0, 'host':17, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1800},
              {'router':0, 'host':16, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2000},
              {'router':0, 'host':15, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':2500},
              {'router':1, 'host':14, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':1000},
              {'router':1, 'host':35, 'isOccupied':0, 'isEve':0, 'isDarth':0, 'maxRate':3000}]

In [56]:
is_multiple = False
if is_multiple:
    num_streams = lambda : choice([1,2,3])
else:
    num_streams = lambda : 0
num_streams()

0

In [2]:
test = open('test.txt', 'w')
a = [1,2,3,4,5,6]
test.write(f"{str(a)}")
test.flush()

In [4]:
print(str(a))

[1, 2, 3, 4, 5, 6]


In [32]:
def djikstras(weight_matrix, source, n):
    visited = [0 for i in range(n)]
#     temp_bw = [0 for i in range(n)]
#     max_bw = sum([sum(i) for i in weight_matrix])
    bw = [0 for i in range(n)]
    routing_table = [0 for i in range(n)]
    bw[source] = float('inf')

    for i in range(n):
        u = find_max(bw, visited, n)
        if u == -1:
            break
        visited[u] = 1
        for j in range(n):
            if bw[j] < min(weight_matrix[u][j], bw[u]):
                routing_table[j] = u
                bw[j] = min(weight_matrix[u][j], bw[u])
    return routing_table, bw

def find_max(bw, visited, n):
    max_index = -1
    max_bw = 0
    for i in range(n):
        if bw[i] > max_bw and visited[i] == 0:
            max_bw=bw[i]
            max_index=i
#             print('max_bw = ',max_bw, 'max_index = ', max_index)
    return max_index

def find_path(alice, bob, requested_rate):
    destination_router = None
    source_router = None
    max_rate = float('inf')
#     print("host_links:")
#     for i in host_links:
#         print(i)
#     min_source_rate = 0
#     min_dest_rate = 0
    for i in host_links:
        if i['host'] == alice and i['maxRate']>0:
            source_router = i['router']
            source_link = i
            max_rate = min(requested_rate, i['maxRate'], max_rate)
#             if i['maxRate'] < max_rate:
#                 max_rate = i['maxRate'] 
#             min_source_rate = i['maxRate']
            
        if i['host'] == bob and i['maxRate']>0:
            destination_router = i['router']
            destination_link = i
            max_rate = min(requested_rate, i['maxRate'], max_rate)
#             if i['maxRate'] < max_rate:
#                 max_rate = i['maxRate']
#             min_dest_rate = i['maxRate']
            
    if destination_router == None or source_router == None:
        print("destination_router", destination_router, "source_router", source_router)
        return None
    else:
        print("found host link")
    
    weight_matrix = [[float('inf') if i == j else 0 for i in range(num_routers)] for j in range(num_routers)]
    for i in router_links:
        if i['maxRate']>0:
            weight_matrix[i['node_a']][i['node_b']] = i['maxRate']
            weight_matrix[i['node_b']][i['node_a']] = i['maxRate']
#     print("weight_matrix:")
#     for i in weight_matrix:
#         print(i)
    
#     print("destination: ", destination_router, "source: ", source_router)
    trail, cost = djikstras(weight_matrix, source_router, num_routers)
#     print('trail: ', trail, 'cost: ', cost)
#     print("rexcieved trail = ", trail)
    if cost[destination_router] == float('inf'):
        print("dist of host infinite")
        return None
    
    current_node = destination_router
    path = []
    while current_node != source_router:
#         print('yo')
        path.insert(0, current_node)
        node_1 = current_node
        current_node = trail[current_node]
        node_2 = current_node
        for i in router_links:
            if ((i['node_a'] == node_1 and i['node_b'] == node_2) or (i['node_a'] == node_2 and i['node_b'] == node_1)) and i not in path:
    #                 i['isOccupied'] = 1
                path.insert(0, i)
                max_rate = min(requested_rate, i['maxRate'], max_rate) 
                break
    
#     minimum_rate = float('inf')
#     for i in path:
#         minimum_rate = min(minimum_rate, required_rate, i['maxRate'])
    
    
#     print("path: ", path)
    path.insert(0, source_router)
#     for i in host_links:
#         if (i['host'] == alice and i['router'] == source_router) and i['isOccupied'] == 0:
#             i['isOccupied'] = 1
#             path.insert(0, i)
            
#         if (i['host'] == bob and i['router'] == destination_router) and i['isOccupied'] == 0:
#             i['isOccupied'] = 1
#             path.append(i)
            
#     source_link['isOccupied'] = 1
    path.insert(0, source_link)
    
#     destination_link['isOccupied'] = 1
    path.append(destination_link)
    
    path.insert(0, alice)
    path.append(bob)
    
    for i in path:
        if not type(i) == int:
            i['maxRate'] -= max_rate
    
#     print("returned paths: ", path)
    return path

initialise_network()
# weight_matrix = [[float('inf') if i == j else 0 for i in range(num_routers)] for j in range(num_routers)]
# for i in router_links:
#     if i['maxRate']>0:
#         weight_matrix[i['node_a']][i['node_b']] = i['maxRate']
#         weight_matrix[i['node_b']][i['node_a']] = i['maxRate']
# print(djikstras(weight_matrix, 1, num_routers))
print(find_path(35, 25, 500))
print(find_path(35, 18, 500))
# djikstras()

found host link
[35, {'router': 1, 'host': 35, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 2500}, 1, {'node_a': 1, 'node_b': 3, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 500}, 3, {'node_a': 3, 'node_b': 4, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 1166}, 4, {'node_a': 4, 'node_b': 6, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 750}, 6, {'node_a': 6, 'node_b': 7, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 928}, 7, {'node_a': 7, 'node_b': 8, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 928}, 8, {'node_a': 8, 'node_b': 11, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 1500}, 11, {'node_a': 12, 'node_b': 11, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 2833}, 12, {'router': 12, 'host': 25, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 1200}, 25]
found host link
[35, {'router': 1, 'host': 35, 'isOccupied': 0, 'isEve': 0, 'isDarth': 0, 'maxRate': 2000}, 1, {'node_a': 1, 'node_b': 3, 'isOccupied': 0, 'isE

In [5]:
rates = [0 for i in range(22)]
for i in range(22):
    rates[i] = choice(list(range(800, 2000)))
print(rates)

[1310, 1017, 1435, 1867, 1373, 1833, 1568, 808, 1529, 1357, 1003, 1213, 820, 1205, 851, 1675, 1891, 1548, 1400, 1089, 1887, 1892]


In [19]:
a = [{'a':1},{'a':2},{'a':3},{'a':4},{'a':5},{'a':6}]
for i in range(2*len(a)):
    a.append(a[i].copy())
print(a)
a[0]['b'] = 1.5
print(a)

[{'a': 1}, {'a': 2}, {'a': 3}, {'a': 4}, {'a': 5}, {'a': 6}, {'a': 1}, {'a': 2}, {'a': 3}, {'a': 4}, {'a': 5}, {'a': 6}, {'a': 1}, {'a': 2}, {'a': 3}, {'a': 4}, {'a': 5}, {'a': 6}]
[{'a': 1, 'b': 1.5}, {'a': 2}, {'a': 3}, {'a': 4}, {'a': 5}, {'a': 6}, {'a': 1}, {'a': 2}, {'a': 3}, {'a': 4}, {'a': 5}, {'a': 6}, {'a': 1}, {'a': 2}, {'a': 3}, {'a': 4}, {'a': 5}, {'a': 6}]


In [5]:
import IPython
a = 'sardar'
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

# restartkernel()
print(a)

sardar
